You can tag `optional parameters` with one of `three` `caller info attributes`  

- `[CallerMemberName]` applies the `caller’s` member `name`.
- `[CallerFilePath]` applies the path to the caller’s `source code file`.
- `[CallerLineNumber]` applies the `line number` in the caller’s source code file.

In [3]:
using System;
using System.Runtime.CompilerServices;

static void Foo (
[CallerMemberName] string memberName = null,
[CallerFilePath] string filePath = null,
[CallerLineNumber] int lineNumber = 0)
{
    Console.WriteLine (memberName);
    Console.WriteLine (filePath);
    Console.WriteLine (lineNumber);
}
void TestCaller() => Foo();

TestCaller();

TestCaller

13


***Caller info attributes*** are useful for `logging`—and for implementing patterns such as firing a single change notification event whenever any property on an object
changes.

In [ ]:
public delegate void PropertyChangedEventHandler

(object sender, PropertyChangedEventArgs e);

public class PropertyChangedEventArgs : EventArgs
{
    public PropertyChangedEventArgs (string propertyName){};
    public virtual string PropertyName { get; }
}

public interface INotifyPropertyChanged
{
    event PropertyChangedEventHandler PropertyChanged;
}

public class Foo : INotifyPropertyChanged
{
    public event PropertyChangedEventHandler PropertyChanged = delegate { };
    void RaisePropertyChanged ([CallerMemberName] string propertyName = null)
        => PropertyChanged (this, new PropertyChangedEventArgs (propertyName));
    string customerName;
    public string CustomerName
    {
        get => customerName;
        set
        {
            if (value == customerName) return;
            customerName = value;
            RaisePropertyChanged();
            // The compiler converts the above line to:
            // RaisePropertyChanged ("CustomerName");
        }
    }
}


### CallerArgumentExpression (C# 10)

In [1]:
Print (Math.PI * 2);

void Print (double number,[CallerArgumentExpression("number")] string expr = null)
    => Console.WriteLine (expr);
    
// Output: Math.PI * 2


Error: (3,28): error CS0246: The type or namespace name 'CallerArgumentExpressionAttribute' could not be found (are you missing a using directive or an assembly reference?)
(3,28): error CS0246: The type or namespace name 'CallerArgumentExpression' could not be found (are you missing a using directive or an assembly reference?)

The **main application** for this feature is when writing `validation` and `assertion` ***libraries***.

In [ ]:
Assert (2 + 2 == 5);

void Assert (bool condition, [CallerArgumentExpression ("condition")] string message = null)
{
    if (!condition) throw new Exception ("Assertion failed: " + message);
}

In [ ]:
//Another example is the static ThrowIfNull method on the ArgumentNullException
//class.

public static void ThrowIfNull (object argument, [CallerArgumentExpression("argument")] string paramName = null)
{
    if (argument == null)
    throw new ArgumentNullException (paramName);
}